# Generate a Powerpoint Slide Deck based on Data in a Database

This is a simple example how a Powerpoint slide deck can be generated using Python. A typical use case is that an overview presentation of all projects needs to be created. The data about the projects is in a database, but instead of manually copying and pasting, the whole process can be automated.

For a good-looking presentation, a template has to be used.



## Step 0: Preparing the Environment

Install the libraries to manipulate Powerpoint files:

In [ ]:
!pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.5 MB/s eta 0:00:00


Mount the Google Drive where the files are and set it as the working directory

In [ ]:
from google.colab import drive

# Mount the Google Drive at mount
mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)

# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/My Drive/Colab Notebooks/PPT Generation"


# Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: Changing directory to  /content/gdrive/My Drive/Colab Notebooks/PPT Generation
/content/gdrive/My Drive/Colab Notebooks/PPT Generation


## Step 1: Setting the parameters

First, let's define the name the Powerpoint template as well as the resulting Powerpoint slide deck.

In [ ]:
ppt_templatename = 'PPT_Template.pptx'
ppt_filename = 'Project Overview.pptx'

Second, define the data from which the slide deck should be generated. Usually this would come from a database, but for simplicity reasons we just hard-code a Pandas dataframe with 4 columns:
* **Project**: Name of the project
* **Description**: A short text describing the project
* **Duration**: A text with from-to dates
* **ImageLink**: A URL to a picture illustrating the project


In [ ]:
import pandas as pd

column_names = ['Project',
           'Description',
           'Duration',
           'ImageLink']
row1 = ['Urban Green Space Revitalization',
        'This project focuses on transforming an underutilized urban area into a vibrant green space that includes community gardens, walking trails, and recreational areas. The goal is to enhance the quality of life for residents by providing a sustainable and accessible natural environment.',
        'January 1, 2023 - December 31, 2023',
        'https://www.acbconsultingservices.com/uploads/optimized/urban-green-space-projects-urban-greening-projects-46-a.jpg']
row2 = ['Renewable Energy Initiative',
        'This project aims to increase the adoption of renewable energy sources within a local community. It involves the installation of solar panels on residential and commercial buildings, as well as educational programs to promote energy efficiency and sustainability.',
        'March 1, 2023 - February 28, 2024',
        'https://media.assettype.com/advait%2F2024-01%2F2a37fc15-f8c9-4c6f-993e-c4568900452d%2FRenewable_Energy_Featured_1.jpg']
row3 = ['Historical Preservation Program',
        'This project is dedicated to preserving and restoring historical buildings in the downtown area. It includes structural repairs, aesthetic enhancements, and the creation of a historical walking tour to educate the public about the city\'s rich heritage.',
        'June 1, 2023 - May 31, 2024',
        'https://bgr.com/wp-content/uploads/2022/03/AdobeStock_194080021.jpeg?resize=1020%2C574&quality=82']

project_data = pd.DataFrame(columns=column_names, data=[row1, row2, row3])

project_data

,Project,Description,Duration,ImageLink
0,Urban Green Space Revitalization,This project focuses on transforming an underu...,"January 1, 2023 - December 31, 2023",https://www.acbconsultingservices.com/uploads/...
1,Renewable Energy Initiative,This project aims to increase the adoption of ...,"March 1, 2023 - February 28, 2024",https://media.assettype.com/advait%2F2024-01%2...
2,Historical Preservation Program,This project is dedicated to preserving and re...,"June 1, 2023 - May 31, 2024",https://bgr.com/wp-content/uploads/2022/03/Ado...


## Step 2: Create the slide deck using the template

The current Python libary for manipulating Powerpoint slides doesn't support the deletion of slides. Hence, the template file shouldn't contain any slides!

Note: The following code doesn't include any error handling.

First we define a function that downloads an image:

In [ ]:
from PIL import Image
from io import BytesIO

def download_image(url):
    # temporary file name
    file_name = 'temp_picture.jpg'

    # Send a GET request to the URL
    response = req.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the image from the response content
        image = Image.open(BytesIO(response.content))
        image.save(file_name )
        return file_name
    else:
        print(f"Failed to retrieve image. HTTP Status code: {response.status_code}")
        return None

Now the rest of the code:


In [ ]:
import pptx
import datetime as dt
import requests as req
from PIL import Image

# Open the template
presentation = pptx.Presentation(ppt_templatename)

# Clean up: Delete all existing slides  --- NOT SUPPORTED YET !!!!
# presentation.slides.clear()

# Get the layouts
layout_title = presentation.slide_master.slide_layouts.get_by_name('Title Slide')
layout_project_description = presentation.slide_master.slide_layouts.get_by_name('1_Comparison')


# Create title slide
title_slide = presentation.slides.add_slide(layout_title)

# As the library is not supporting the look up of shapes by name, we define the indexes here
shape_title = 0
shape_subtitle = 1

#Insert title slide text
title_slide.shapes[shape_title].text = 'Project Descriptions'
title_slide.shapes[shape_subtitle].text = dt.datetime.today().strftime('%B %Y')

# Iterate through data and create one slide per project
for idx in project_data.index:
  print(project_data['Project'][idx], ' ', project_data['ImageLink'][idx])
  # Create project slide
  project_slide = presentation.slides.add_slide(layout_project_description)

  # As the library is not supporting the look up of shapes by name, we define the indexes here
  prj_name = 0
  prj_desc = 1
  prj_duration = 2
  prj_image = 3


  #Insert title slide text
  project_slide.shapes[prj_name].text = project_data['Project'][idx]
  project_slide.shapes[prj_desc].text = project_data['Description'][idx]
  project_slide.shapes[prj_duration].text = project_data['Duration'][idx]
  illustration = project_slide.shapes[prj_image]
  illustration.insert_picture(download_image(project_data['ImageLink'][idx]))

# Save and close presentation
presentation.save(ppt_filename)

Urban Green Space Revitalization   https://www.acbconsultingservices.com/uploads/optimized/urban-green-space-projects-urban-greening-projects-46-a.jpg
Renewable Energy Initiative   https://media.assettype.com/advait%2F2024-01%2F2a37fc15-f8c9-4c6f-993e-c4568900452d%2FRenewable_Energy_Featured_1.jpg
Historical Preservation Program   https://bgr.com/wp-content/uploads/2022/03/AdobeStock_194080021.jpeg?resize=1020%2C574&quality=82
